# Importando bibliotecas

Além dos importes tem alguns sets de configurações do notebook

In [3]:
import pandas as pd
import numpy as np

# Define a opção para mostrar TODAS as colunas, sem limite
pd.set_option('display.max_columns', None)

# Define a opção para mostrar TODAS as linhas, sem limite
pd.set_option('display.max_rows', None)


# Define a opção para mostrar o número máximo de elementos do array como infinito (None)
np.set_printoptions(threshold=np.inf)

upload do dataset completo

In [4]:
df = pd.read_csv("../datasets/dataframe_yuri_xiaomi_x5.csv")
df.shape

(2090, 20)

Separações dos dados de acordo com as direções

In [5]:
df_norte = df[df['Direction'] == 'norte']
df_leste = df[df['Direction'] == 'leste']
df_oeste = df[df['Direction'] == 'oeste']
df_sul = df[df['Direction'] == 'sul']
print(f"Shape df_norte: {df_norte.shape}\nShape df_leste: {df_leste.shape}\nShape df_oeste: {df_oeste.shape}\nShape df_sul: {df_sul.shape}")

Shape df_norte: (521, 20)
Shape df_leste: (520, 20)
Shape df_oeste: (522, 20)
Shape df_sul: (527, 20)


Importando bibliotecas para o treinamento do KNN

In [6]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import random

# Treinamento Norte X Teste Outras direções (LESTE, OESTE, SUL)

Separando os dados em treinamento

In [ ]:
df_filtrado_norte =  df_norte.drop(columns = ['Direction', 'Alpha', 'Beta', 'Gama', 'Device', 'Brand', 'Created', 'Model' ])


X_train_norte = df_filtrado_norte.drop(columns = ['X', 'Y', 'RP']).to_numpy()
Y_train_norte = df_filtrado_norte["RP"].to_numpy()


print(X_train_norte.shape)
print(Y_train_norte.shape)

(521, 9)
(521,)


Separando os dados de teste/validação

In [25]:
#conjunto de dados de teste para iterar e fazer o teste, após o treinamento com os dados de direção norte
conjuntos_teste = {'leste':'', 'oeste':'', 'sul':''}

df_filtrado_leste = df_leste.drop(columns = ['Direction', 'Alpha', 'Beta', 'Gama', 'Device', 'Brand', 'Created', 'Model' ])
df_filtrado_oeste = df_oeste.drop(columns = ['Direction', 'Alpha', 'Beta', 'Gama', 'Device', 'Brand', 'Created', 'Model' ])
df_filtrado_sul = df_sul.drop(columns = ['Direction', 'Alpha', 'Beta', 'Gama', 'Device', 'Brand', 'Created', 'Model' ])


X_test_leste =  df_filtrado_leste.drop(columns = ['X', 'Y', 'RP']).to_numpy()
X_test_oeste = df_filtrado_oeste.drop(columns = ['X', 'Y', 'RP']).to_numpy()
X_test_sul = df_filtrado_sul.drop(columns = ['X', 'Y', 'RP']).to_numpy()

Y_test_leste = df_filtrado_leste['RP'].to_numpy()
Y_test_oeste = df_filtrado_oeste['RP'].to_numpy()
Y_test_sul = df_filtrado_sul['RP'].to_numpy()


conjuntos_teste['leste'] = {'X':X_test_leste, 'Y':Y_test_leste}
conjuntos_teste['oeste'] = {'X':X_test_oeste, 'Y':Y_test_oeste}
conjuntos_teste['sul'] = {'X':X_test_sul, 'Y':Y_test_sul}

print(conjuntos_teste)


{'leste': {'X': array([[ -73,  -76,  -62,  -74,  -81,  -74, -105,  -73,  -84],
       [ -73,  -83,  -62,  -78,  -82,  -84,  -83,  -73, -105],
       [ -69,  -71,  -63,  -80,  -84,  -82,  -79,  -74,  -83],
       [ -68,  -75,  -66,  -85,  -82,  -83,  -80,  -73,  -86],
       [ -72,  -77,  -65,  -82,  -84,  -78,  -80,  -75,  -94],
       [ -72,  -78,  -65,  -82,  -82, -105,  -85,  -72,  -83],
       [ -73,  -79,  -66,  -81,  -84,  -80,  -81,  -72,  -82],
       [ -72,  -77,  -66,  -78,  -81,  -88,  -81,  -72,  -84],
       [ -76,  -77,  -64,  -78,  -82,  -87,  -80,  -72,  -82],
       [ -75,  -78,  -65,  -83,  -82,  -79,  -85,  -72,  -83],
       [ -78,  -74,  -66,  -80,  -82,  -78,  -79,  -75,  -87],
       [ -78,  -80,  -65,  -80,  -90, -105,  -86,  -72,  -83],
       [ -76,  -77,  -62,  -77,  -84, -101,  -82,  -73,  -82],
       [ -76,  -76,  -67,  -81,  -82,  -80,  -88,  -82,  -83],
       [ -70,  -73,  -63,  -82,  -88,  -80,  -81,  -78,  -84],
       [ -72,  -77,  -66,  -81,  -83,  

# Funções para conversao para posição X Y

Mapeamento das posicoes para valores X Y

In [26]:
df_rp = df["RP"].to_numpy()
categories = np.unique(df_rp)
print(df_rp.shape)

sets_positions = {'X':{}, "Y":{}}

for categorie in categories:
    
    first_record_category = df.loc[df['RP'] == categorie].iloc[0]
    
    
    y = float(first_record_category["Y"])
    x = float(first_record_category["X"])


    split_categorie = categorie.split("_")
    #print(split_categorie)

    if(split_categorie[1] not in sets_positions["Y"]):
        #print(f"Adicionando o valor {y} associado a P_{split_categorie[1]}_x")
        sets_positions["Y"][f"{split_categorie[1]}"] = y

    if(split_categorie[2] not in sets_positions["X"]):
        #print(f"Adicionando o valor {x} associado a P_Y_{split_categorie[2]}")
        sets_positions["X"][f"{split_categorie[2]}"] = x
    


print(sets_positions["X"])
print(sets_positions["Y"])

(2090,)
{'1': 1.57, '2': 3.92, '3': 6.27}
{'1': 0.77, '2': 2.03, '3': 3.76, '4': 5.42, '5': 7.09, '6': 8.75, '7': 10.42, '8': 12.08, '9': 13.74}


In [27]:
def converte_categoria_posicao(values_pred,values_real):
    posicoes_pred = []
    posicoes_real = []
    shape_pred = values_pred.shape[0]
    shape_real = values_real.shape[0]

    if(shape_pred != shape_real):
        print(f"Erro a dimensão dos valores previstos é diferente da dimensão dos valores reais: (pred {values_pred}) (real {values_real})")
        return
    
    for index in range(len(values_real)):

        value_pred_split = values_pred[index].split("_")
        value_real_split = values_real[index].split("_")

        x_pred = sets_positions['X'][value_pred_split[2]]
        y_pred = sets_positions['Y'][value_pred_split[1]]
        x_real = sets_positions['X'][value_real_split[2]]
        y_real = sets_positions['Y'][value_real_split[1]]

        posicoes_pred.append((x_pred, y_pred))
        posicoes_real.append((x_real, y_real))

    return posicoes_pred, posicoes_real

# Treinamento e Teste dos modelos

Iniciando o treinamento com os parametros dos modelos

In [28]:
cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state=42)
param_grid = {'n_neighbors':[1,3,5,7,9,11]}
knn_clf = KNeighborsClassifier()

In [35]:
k_valores = [1, 3, 5, 7, 20]

conjunto_test_reais_preditos = {'leste':'', 'oeste':'', 'sul':''}

'''
conjuntos_teste['leste'] = {'X':X_test_leste, 'Y':Y_test_leste}
conjuntos_teste['oeste'] = {'X':X_test_oeste, 'Y':Y_test_oeste}
conjuntos_teste['sul'] = {'X':X_test_sul, 'Y':Y_test_sul}
'''

for k in k_valores:
    knn_clf = KNeighborsClassifier(n_neighbors=k)
    print(f"======================= Treinando o modelo KNN com k={k} =======================")
    print(f'SHAPE X_TRAIN_NORTE: {X_train_norte.shape}.')
    print(f'SHAPE Y_TRAIN_NORTE: {Y_train_norte.shape}.')
    knn_clf.fit(X_train_norte, Y_train_norte)
    
    for direcao, dados in conjuntos_teste.items():
        print(f"Começando o teste da direção {direcao} ")
        # Predizer os valores:
        y_pred = knn_clf.predict(dados['X'])
        y_reais = dados['Y']
        
        
        #Guardar em uma lista os valores preditos e reais de cada direção: 
        conjunto_test_reais_preditos[direcao] = [y_reais, y_pred]
        
        

    for direcao, dados in conjunto_test_reais_preditos.items():
        print(f'~~~~~~~~ INICIANDO OS RESULTADOS PARA A DIREÇÃO = {direcao} e K = {k} ~~~~~~~~ ')
        
        # conversao das posicoes RP em X e Y
        positions_pred, positions_test = converte_categoria_posicao(values_pred=dados[1], values_real=dados[0])
        
        preditos = np.array(positions_pred)
        reais = np.array(positions_test)
        
        #Calculo do erro medio da distancia euclidiana entre os pontos
        distancias = np.sqrt((preditos[:, 0] - reais[:, 0])**2 + (preditos[:, 1] - reais[:, 1])**2)
        mae = np.mean(distancias)
        
        # Analise dos resultados
        print(f"\nErro médio absoluto da distância euclidiana entre os valores preditos e os valores reais: {mae:.2f}")

        print(f'Y_test_{direcao}_real: {reais.shape}')
        print(f'Y_test_{direcao}_predito: {preditos.shape}')
        print(classification_report(y_true=dados[0], y_pred=dados[1]))

        
        
    
    

======================= Treinando o modelo KNN com k=1 =======================
SHAPE X_TRAIN_NORTE: (521, 9).
SHAPE Y_TRAIN_NORTE: (521,).
Começando o teste da direção leste 
Começando o teste da direção oeste 
Começando o teste da direção sul 
~~~~~~~~ INICIANDO OS RESULTADOS PARA A DIREÇÃO = leste e K = 1 ~~~~~~~~ 

Erro médio absoluto da distância euclidiana entre os valores preditos e os valores reais: 2.92
Y_test_leste_real: (520, 2)
Y_test_leste_predito: (520, 2)
              precision    recall  f1-score   support

       P_1_1       0.04      0.10      0.06        29
       P_1_2       0.00      0.00      0.00        29
       P_1_3       0.49      0.79      0.61        29
       P_2_1       0.00      0.00      0.00        29
       P_2_2       0.00      0.00      0.00        29
       P_2_3       0.00      0.00      0.00        29
       P_3_1       0.16      0.18      0.17        17
       P_3_2       0.06      0.06      0.06        16
       P_3_3       0.50      0.06      

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag